# Models Used in this Python Notebook:
    - BenFastText+SVM
    - BenFastText+LSTM

In [2]:
import pandas as pd
import numpy as np
import re
import unicodedata
import nltk
import matplotlib.pyplot as plt
import string
import re
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec
from gensim.models import FastText
from sklearn.feature_extraction.text import CountVectorizer
import sys
from collections import Counter
import sys
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras.initializers import Constant

#### Importing Dataset

In [3]:
df=pd.read_csv("Bengali_hate_speech.csv")

#### removing emojis

In [4]:
def emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [5]:
df['sentence']=df['sentence'].apply(emoji)

#### removing punctutaions

In [6]:
def punctuation(t1):
    whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)
    bangla_fullstop = u"\u0964"
    punctSeq   = u"['\"“”‘’]+|[.?!,…]+|[:;]+"
    punc = u"[(),$%^&*+={}\[\]:\"|\'\~`<>/,¦!?½£¶¼©⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞⅟↉¤¿º;-]+"
    t1= whitespace.sub(" ",t1).strip()
    t1 = re.sub(punctSeq, " ", t1)
    t1 = re.sub(bangla_fullstop, " ",t1)
    t1 = re.sub(punc, " ", t1)
    return t1

In [7]:
df['sentence']=df['sentence'].apply(punctuation)

#### replacing bengali numbers into english numbers

In [8]:
def replace_num(one):
    one = one.replace("০","0")
    one=one.replace("১","1")
    one=one.replace("২","2")
    one=one.replace("৩","3")
    one=one.replace("৪","4")
    one=one.replace("৫","5")
    one=one.replace("৬","6")
    one=one.replace("৭","7")
    one=one.replace("৮","8")
    one=one.replace("৯","9")
    return one

In [9]:
df['sentence']=df['sentence'].apply(replace_num)

#### removing non-bengali alphabets & numbers

In [10]:
def non_bengali(a):
    a = "".join(i for i in a if i in [".","।"] or 2432 <= ord(i) <= 2559 or ord(i)== 32)
    a=re.sub(' +', ' ', a)
    return a

In [11]:
df['sentence']=df['sentence'].apply(non_bengali)

In [12]:
def basic_clean(text):    
    words = text.split()
    return words

In [13]:
values=[]
size=df.shape[0]
for i in range(0,size):
        worddd=basic_clean(df['sentence'][i])
        values.append(worddd)     

In [14]:
words=[]
for val in values:
    words.extend(val)

In [14]:
len(words)

437558

In [34]:
import fasttext

In [38]:
import gensim

In [40]:
import torch

In [ ]:
/Users/rajdeepdas/Downloads/cc.bn.300.vec
model = fasttext.load_model("/Users/rajdeepdas/Downloads/cc.bn.300.vec")

### Importin Bengali FastText Model

In [43]:
model = fasttext.load_model("/Users/rajdeepdas/Downloads/cc.bn.300.bin")

In [54]:
model.get_subwords('খেলা') 

(['খেলা', '<খেলা', 'খেলা>'], array([    546, 2610156, 3129338]))

In [69]:
docs_vectors = pd.DataFrame()
i=0
for doc in df['sentence']:
    if(i%100==0):
        print(i)
    i=i+1
    temp = pd.DataFrame()
    for word_a in doc.split():
        try:
            word_vec=model[word_a]
            temp=temp.append(pd.Series(word_vec),ignore_index=True)  
        except:
            pass
    doc_vector = temp.mean()
    docs_vectors=docs_vectors.append(doc_vector,ignore_index=True)
    
docs_vectors['hate'] = df['hate']
docs_vectors = docs_vectors.dropna()
train_x, test_x, train_y, test_y = train_test_split(docs_vectors.drop('hate', axis = 1),
                                                   docs_vectors['hate'],
                                                   test_size = 0.2,
                                                   random_state = 1)   

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [70]:
train_x[1]

6141     0.030670
4021    -0.037245
5877    -0.037394
10869   -0.037750
8857    -0.035911
           ...   
17300    0.001609
5192     0.011513
12182    0.005344
235      0.000329
29804    0.031361
Name: 1, Length: 23943, dtype: float32

### SVM Classifier

In [71]:
clf = svm.SVC(kernel='linear') 
clf.fit(train_x, train_y)

#Predict the response for test dataset
test_pred = clf.predict(test_x)
accuracy=accuracy_score(test_y, test_pred)
print('Accuracy: %f' % accuracy)

Accuracy: 0.811894


### LSTM Model

In [72]:
max_length = max([len(s.split()) for s in df['sentence']])

In [73]:
# Count all the words using Counter Method
##
words=[]
for val in values:
    words.extend(val)
count_words = Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}
reviews_int = []
for val in values:
    r = [vocab_to_int[w] for w in val]
    reviews_int.append(r)
print (reviews_int[0:3])
review_pad=pad_sequences(reviews_int,maxlen=max_length)
sentiment=df['hate'].values

[[3061, 67, 292, 12658], [67, 97, 171, 938, 804, 54], [7013, 338, 301, 167, 172, 68, 363, 15, 50, 7, 75, 32, 124, 76, 13, 1742, 1319, 126, 1942, 14]]


In [76]:
num_words=47445

In [77]:
embedding_matrix=np.zeros((len(sorted_words)+1, 300))
for word, i in sorted_words:
    if i > num_words:
        continue
    try:    
        embedding_vector = model[word]
    except:
        pass
    if embedding_vector is not None:
        embedding_matrix[i]= embedding_vector

In [88]:
modell=Sequential()
embedding_layer = Embedding(num_words, 300,input_length=max_length)
modell.add(embedding_layer)
modell.add(LSTM(units=100,dropout=0.2,recurrent_dropout=0.2))
modell.add(Dense(1,activation='sigmoid'))

modell.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [89]:
VALIDATION_SPLIT=0.2
indices=np.arange(review_pad.shape[0])
np.random.shuffle(indices)
sentiment=sentiment[indices]
num_validation_samples=int(VALIDATION_SPLIT*review_pad.shape[0])
x_trainn_pad=review_pad[:-num_validation_samples]
y_trainn=sentiment[:-num_validation_samples]
x_testt_pad=review_pad[-num_validation_samples:]
y_testt=sentiment[-num_validation_samples:]

In [90]:
modell.fit(x_trainn_pad,y_trainn,validation_data=(x_testt_pad,y_testt),batch_size=64,epochs=5)

Epoch 1/5
291/375 [======================>.......] - ETA: 54:38 - loss: 0.6450 - accuracy: 0.6620

KeyboardInterrupt: 

In [84]:
test_pred = modell.predict(x_testt_pad)
accuracy=accuracy_score(y_testt, test_pred.round())
print('Accuracy: %f' % accuracy)

Accuracy: 0.574667
